In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [64]:
from bot import sendMessage
    
from __future__ import print_function
import seaborn; seaborn.set()

In [4]:
r_train = pd.read_csv('data/r_train.csv', index_col=0)
r_test = pd.read_csv('data/r_test.csv', index_col=0)
r_target = pd.read_csv('data/target.csv', index_col=0)['0']

In [140]:
droplist = set(['ind_var6', 'saldo_medio_var17_hace3', 'delta_imp_trasp_var33_out_1y3', 
                'saldo_medio_var17_hace2', 'ind_var20_0', 'delta_imp_reemb_var33_1y3',
                'delta_imp_reemb_var13_1y3', 'num_var13_medio_0', 'ind_var20', 'ind_var18_0',
                'delta_imp_amort_var18_1y3', 'delta_imp_amort_var34_1y3', 'saldo_medio_var12_ult1',
                'delta_imp_aport_var13_1y3', 'delta_imp_venta_var44_1y3', 'saldo_medio_var17_ult3',
                'num_meses_var13_medio_ult3', 'delta_num_aport_var33_1y3', 'delta_num_aport_var17_1y3',
                'delta_num_compra_var44_1y3', 'imp_reemb_var33_ult1', 'delta_imp_trasp_var17_out_1y3',
                'imp_reemb_var17_hace3', 'ind_var34_0', 'delta_imp_trasp_var17_in_1y3'])

droplist = ['imp_op_var39_efect_ult3', 'num_op_var41_ult3', 'imp_op_var39_efect_ult1', 
            'delta_imp_aport_var13_1y3', 'delta_imp_reemb_var33_1y3', 'num_trasp_var33_out_ult1', 
            'ind_var18_0', 'delta_imp_reemb_var13_1y3', 'saldo_medio_var13_medio_ult3', 
            'ind_var20_0', 'imp_trasp_var17_in_hace3', 'num_op_var41_ult1', 'num_var1_0', 
            'num_var44', 'ind_var40_0', 'ind_var13_largo_0', 'num_var34_0', 'ind_var24', 
            'num_op_var41_efect_ult1', 'saldo_var24', 'imp_op_var41_ult1', 'ind_var20', 
            'ind_var39_0', 'num_var40', 'num_trasp_var17_in_ult1', 'num_var8_0', 
            'saldo_medio_var17_ult3', 'ind_var6_0', 'num_var8', 'num_var6', 'num_var13_medio_0', 
            'num_var4', 'ind_var14', 'num_var1', 'saldo_var1', 'saldo_medio_var33_ult3', 
            'num_reemb_var17_hace3', 'num_op_var41_efect_ult3', 'delta_num_venta_var44_1y3', 
            'delta_num_aport_var33_1y3', 'saldo_medio_var17_hace3', 'saldo_medio_var17_hace2', 
            'ind_var34_0', 'saldo_medio_var33_ult1', 'num_var45_ult3', 'saldo_medio_var12_ult1', 
            'num_var18_0', 'saldo_medio_var13_medio_hace2', 'num_reemb_var33_ult1', 'ind_var13_corto_0', 
            'ind_var13', 'num_op_var41_hace3', 'num_op_var41_hace2', 'ind_var7_emit_ult1', 
            'num_var12', 'saldo_medio_var13_largo_ult1', 'ind_var24_0', 'saldo_var17', 
            'num_var5', 'delta_num_compra_var44_1y3', 'saldo_var26', 'saldo_var34', 
            'saldo_medio_var13_corto_ult1', 'saldo_var13_corto', 'saldo_var18', 
            'delta_num_aport_var17_1y3', 'num_meses_var13_medio_ult3', 'num_trasp_var17_out_ult1', 
            'ind_var44_0', 'imp_trasp_var33_out_ult1', 'num_var40_0', 'ind_var1', 'num_var13_corto']
#droplist = []

In [141]:
data = r_train.drop(droplist, axis=1)

In [142]:
%%time
dc = data.corr()

Wall time: 12.7 s


In [143]:
#Find correlations
def bad_feature_pairs(corr, columns, th):
    bad_fts = set()
    bad_count = {ft:0 for ft in columns}
    for ft1 in columns:
        for ft2 in columns:
            if ft1 == ft2: 
                continue
            if corr[ft1][ft2] > th:
                pair = tuple(sorted([ft1, ft2]))
                if pair not in bad_fts:
                    bad_fts.add(pair)
                    bad_count[ft1] += 1
                    bad_count[ft2] += 1
    return bad_fts, bad_count

corr_th = .98
bad_fts, bad_count = bad_feature_pairs(dc, dc.columns, corr_th)
print(len(bad_fts))

0


In [144]:
for ft1, ft2 in sorted(list(bad_fts), key=lambda x: -dc[x[0]][x[1]]):
    print(ft1, ft2, dc[ft1][ft2])

In [145]:
for ft in sorted(bad_count, key=lambda x: -bad_count[x]):
    if bad_count[ft] > 0:
        print(ft, bad_count[ft])

In [146]:
correlated = set(map(lambda x: x[0], bad_fts) + map(lambda x: x[1], bad_fts))
len(correlated)

0

In [148]:
to_drop = set()
while True:
    _, bc = bad_feature_pairs(dc, set(dc.columns).difference(to_drop), corr_th)
    #print(bc)
    bcs = sorted(bc, key=lambda x: -bc[x])
    print(bcs[0], bc[bcs[0]])
    if bc[bcs[0]] == 0:
        break
    to_drop.add(bcs[0])
print(len(to_drop), to_drop)

num_var45_hace2 0
0 set([])
